In [25]:
import os
import requests
import pickle
import pandas as pd
from pathlib import Path
from holoviews import hvplot
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

ImportError: cannot import name 'hvplot' from 'holoviews' (/Users/michael1/opt/anaconda3/envs/alpacaenv/lib/python3.7/site-packages/holoviews/__init__.py)

In [6]:
load_dotenv()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [16]:
crypto = ('ETH', 'XRP', 'ADA', 'DOT', 'SOL')
model = pickle.load(open(Path('../Resources/model.pkl'),'rb'))
scaler = pickle.load(open(Path('../Resources/scaler.pkl'), 'rb'))

In [23]:
for x in crypto:
    data = api.get_crypto_bars(
        f'{x}USD',
        '1Day',
        start=start_date,
        end=end_date
    ).df
    data['actual_returns'] = data['close'].pct_change()
    data['sma_fast'] = data['close'].rolling(window=3).mean()
    data['sma_slow'] = data['close'].rolling(window=10).mean()
    data[['sma_fast', 'sma_slow']] = data[['sma_fast', 'sma_slow']].shift()
    data = data.dropna()
    X_scaled = scaler.transform(X)
    y = model.predict(X_scaled)

    predictions_df = pd.DataFrame(index=X.index)
    predictions_df["predicted_signal"] = y
    predictions_df["actual_returns"] = data["actual_returns"]
    predictions_df["trading_algorithm_returns"] = (
        predictions_df["actual_returns"] * predictions_df["predicted_signal"]
    )
    x = (1 + predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().hvplot()


ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [7]:
start_date = pd.Timestamp("2014-01-01", tz="Australia/Melbourne").isoformat()
end_date = pd.Timestamp("2022-01-01", tz="Australia/Melbourne").isoformat()

data = api.get_crypto_bars(
    'BTCUSD',
    '1Day',
    start=start_date,
    end=end_date
).df
data['actual_returns'] = data['close'].pct_change()
data['sma_fast'] = data['close'].rolling(window=3).mean()
data['sma_slow'] = data['close'].rolling(window=10).mean()

data.head(10)

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2015-01-07 06:00:00+00:00,CBSE,360.00,360.00,264.81,275.44,6.420000,634,298.877741
2015-01-08 06:00:00+00:00,CBSE,297.08,314.91,265.50,288.99,3.930000,393,289.251781
2015-01-12 06:00:00+00:00,CBSE,260.00,260.00,260.00,260.00,1.000000,1,260.000000
2015-01-13 06:00:00+00:00,CBSE,200.00,221.00,200.00,219.63,0.030000,3,213.543333
2015-01-14 06:00:00+00:00,CBSE,220.00,220.00,109.87,120.00,11.274638,1115,155.605993
2015-01-15 06:00:00+00:00,CBSE,199.00,224.00,199.00,204.22,1.775121,165,206.555271
2015-01-16 06:00:00+00:00,CBSE,200.12,209.82,196.51,199.46,2.490000,249,203.132570
2015-01-17 06:00:00+00:00,CBSE,195.00,210.00,184.00,184.00,0.089000,9,195.775281
2015-01-18 06:00:00+00:00,CBSE,210.00,225.51,210.00,225.51,0.410000,5,210.378293
